# Digit Recodnition Using Keras with TensorFlow Backend
        Student Name:    Timothy Cassidy
        Student Number:  G00333333

## Table of Contents

-  [Introduction](#intro)<br>
-  [Imported Modules](#imports)<br>
-  [Building the Neural Network](#buildingthenet)<br>
-  [Findings](#findings)<br>
   -  [Optimizer](#optimizer)
   -  [Epochs](#epochs)
-  [Sources](#sources)


## Introduction

The purpose of this notebook is to outline the various design decisions made when creating the 'digitrec.py' digit recognition script contained within this repository, the purpose of that script is to predict what digit is contained within an image supplied to the neural network. The neural network defined within the script has been trained using the MNIST dataset, this dataset is explained in the Jupyter Notebook titled 'mnist-dataset' which can be found inside this repository, in this case the MNIST dataset is imported using Keras, this is purely for convenience, the dataset used in both case is exactly the same. 

## Imported Modules <a id="imports"></a>

The first imports used are the [mathplotlib.pyplot](https://matplotlib.org/api/pyplot_api.html) and the [numpy](https://docs.scipy.org/doc/numpy/index.html) modules. The purpose of the pyplot import was to ensure the dataset had been imported correctly, this was done by plotting a sample of the images contained within the MNIST dataset. The numpy module was imported to allow a seed to be defined, this ensures reproducibility.

In [ ]:
# import matplotlib.pyplot as plt
import numpy as np

Below are the various Keras imports used. The first import is the MNIST dataset. The second import defines the type of model we will be using, in this case it is the [sequential model](https://keras.io/getting-started/sequential-model-guide/), this model allows us to declare layers and add them sequentially, or one after another. The next few imports are layer modules, these are [Flatten](https://keras.io/layers/core/#flatten), [Conv2D](https://keras.io/layers/convolutional/#conv2d), [MaxPooling2D](https://keras.io/layers/pooling/#maxpooling2d), [Dense](https://keras.io/layers/core/#dense) and [Dropout](https://keras.io/layers/core/#dropout). 

In [ ]:
# keras imports
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils
from keras import backend as K
from keras.preprocessing import image

The [os.path](https://docs.python.org/3/library/os.path.html) module allows access to the systems file system. 

In [ ]:
import os.path as path

The [tkinter]() module provides a windowed view/access to the file system, this is used below to provide individual images to the neural network and make predictions based on them.

In [ ]:
import tkinter as tk
from tkinter import filedialog

## Building the Neural Network <a id="buildingthenet"></a>

The first line below, *K.set_image_dim_ordering('th'), specifies which dimension ordering convention Keras will follow.
In this case the 'th' assumes that the 2D(image) data will be ordered as channels, rows then cols, the other option that could have been used here is 'tf' which assumes the ordering as rows, cols then channels.<br> 
The variable h5FileName stores the name associated with the weights file to be output/input.

In [ ]:
# initialization
K.set_image_dim_ordering('th')
# file name constant
h5FileName = "weights.h5"

Below the MNIST dataset is loaded and X and Y training and testing arrays are populated. Both the X_train and X_test contain the image data and the Y_train and Y_test contain the labels of the image data. The X_train and X_test image arrays are reshaped, this new array shape will be the accepted input of this neural network. The random seed is set to 7, this is set to ensure reproducibility.

In [ ]:
# load data
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
# reshape to be [samples][pixels][width][height]
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')
	
# setting a fixed random seed for reproducibility
seed = 7
np.random.seed(seed)

The inputs are normalized in order to standardize them, this makes can make training faster and can reduce the likelyhood of getting stuck in [local optima](https://www.lindo.com/doc/online_help/lingo17_0/local_optima_vs__global_optima.htm).

In [ ]:
# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255

One hot code outputs are used in order to format the outputs as a binary class matrix.

In [ ]:
# one hot encode outputs
Y_train = np_utils.to_categorical(Y_train)
Y_test = np_utils.to_categorical(Y_test)
num_classes = Y_test.shape[1]

Below is the model definition function. First we state the model used is a sequential model, next the various layers used are added to the model. These are explained below:
<br><br>Conv2D:
        
    This layer is the input layer, here the input shape is defined. I used the activation function 'relu'(Rectified Linear Unit) here because it is widely used and it has the ability to account for non-linearities and interactions. This activation function returns a 0 if a negative value is encountered.
MaxPooling2D:
    
    The objective of the MaxPooling2D layer is to down-sample the input image by reducing it's dimensions and allowing assumptions to be made about it's features, it also reduces the computational cost because there are less dimensions to observe and it's helps to reduce over-fitting.
Dropout:

    The dropout layer is another method of reducing over-fitting of data from the dataset, here I have specified a dropour rate of 0.2, so 20% of the neurons will dropped.
Flatten:

    The flatten layer, which is passed no arguments, reduces the input array to a single dimension (e.g one row), this is very similar to flatten() function of the numpy package.
Dense:

    The dense layer is just a regular layer of neurons, each neuron in this layer recieves input from every neuron in the previous layer, hence the name dense. In this neural net I have defined the last 2 layers as dense layers. The first dense layer is fitted with the 'relu' activation function, which is defined above. 
    The second dense layer is also the output layer, this layer makes use of the one hot encoding defined above and uses the 'softmax' activation function, 'softmax' is not a traditional activation function as it produces multiole outputs(e.g an array).
Finally the model is compiled using categorical_crossentrophy, the 'adam' optimizer along with accuracy metrics, then the created model is returned.

In [ ]:
# define the model
def modelDefn():
	# create model
    model = Sequential()
    # layer definitions
    model.add(Conv2D(32, (5, 5), input_shape=(1, 28, 28), activation='relu'))
    model.add(MaxPooling2D(pool_size=(3, 3)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
    return model

The function below is used to make a prediction on an image input. The function first loads in the image from a URL. Next the image is cast to an array, the dimension of this array have to be expanded in order to be accepted as a valid input in this neural network. Finally the image is passed into the model and a prediction is made, this is then output to the console.

In [ ]:
def prediction(imageURL):
    prediction = ""
    test_image = image.load_img(imageURL, target_size = (28, 28), color_mode="grayscale")
    test_image = np.array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    test_image = np.expand_dims(test_image, axis = 0)
    result = model.predict(test_image)
    
    print(result)

Below is a function call to define the model. Next, using the os.path module, a check is preformed to see if the file name we defined above exists in the current directory, if it does those weights are loaded into the model. If no such file exists the model trained on the MNIST data and the weights are saved inthe current directory. There are six pre-compiled weights files(.h5) contained within this repository, these can be renamed to "weights.h5" if you wish to use them.

In [ ]:
# build the model
model = modelDefn()

# check to see if the weights file exists
if path.isfile(h5FileName):
    print("Loading pre-compiled Neural Net Weights")
    model.load_weights(h5FileName)
else:
    print("Creating Neural Net and saving Weights to file")
    # Fit the model
    model.fit(X_train, outputs, validation_data=(X_test, Y_test), epochs=12, batch_size=200, verbose=2)
    model.save_weights(h5FileName)

The following code is an evaluation of the model using the test data, this outputs a basline error precentile. <br>

In [ ]:
# Final evaluation of the model
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Baseline Error: %.2f%%" % (100-scores[1]*100))

Below we make use of the tkinter module which allows us to select an image from a windowed file system view. This is contained within a while loop. Within the while loop the filepath of the selected image is passed to the prediction model, if no file is selected the loop exits and the application terminates.

In [ ]:
# loop until the program is exited
while True:
    # open a new file explorer window
    root = tk.Tk()
    root.withdraw()

    try:
        # store the url of the selected file
        file_path = filedialog.askopenfilename()
        # call the prediction method of our model
        prediction(file_path)

    except:
        print("No file selected. Exiting application!")
        break

## Findings <a id="findings"></a>

### Optimizer <a id="optimizer"></a>
I had first used 'sgd' as an optimizer, the image below is the console output of the 'sgd' optimizer. While a baseline error of 4.28% is good I achieved much better results using the 'adam' optimizer.

![sgdOptimizer](resultImages/sgdOptimizer.PNG)

The 'adam' optimizer console output with a baseline error of 0.96%, this is less prone to error than the previous optimizer.

![adamOptimizer](resultImages/adamOptimizer.PNG)

### Epochs <a id="epochs"></a>
The number of epochs, which are defined as "*a measure of the number of times all of the training vectors are used once to update the weights.*" [source](https://www.quora.com/What-is-epochs-in-machine-learning). Have an effect on the accuracy of the neural network. I found that 12 epochs was the optimal number in this case, any less and the accuracy was negatively effected. This is shown from the console outputs below.

![adamOptimizer10Epochs](ResultImages/adamOptimizer10Epochs.PNG)
![adamOptimizer11Epochs](ResultImages/adamOptimizer11Epochs.PNG)
![adamOptimizer12Epochs](ResultImages/adamOptimizer12Epochs.PNG)

Although the accuracy increased when increasing the epochs to 13 or 14(console outputs below), this seemed to cause some overfitting of the dataset. This was evident from testing the neural network with images not contained within the MNIST dataset, these images are contained within the Test_Images folder in this repository.

![adamOptimizer13Epochs](resultImages/adamOptimizer13Epochs.PNG)
![adamOptimizer14Epochs](resultImages/adamOptimizer14Epochs.PNG)

### Overfitting <a id="overfitting"><a/>
Overfitting occurs when patterns in the dataset used to train the neural network are focused on too much, this prevents the neural network from generalizing patterns in the training data and applying them to new inputs. The measures taken to prevent this are the use of the MaxPooling2D and the Dropout layers of the model and also reducing the number of epochs seemed to reduce this overfitting. While these measures were taken in order to prevent overfitting, when testing the neural netwok by inputting images not contained within the MNIST dataset, there seems to an issue when trying to differentiate the digit '6' from the digit '8'. This is something that needs to be improved, but for the purpose of this notebook it is an acceptable flaw.


## Sources <a id="sources"></a>
<br>[Rectified Linear Unit](https://www.kaggle.com/dansbecker/rectified-linear-units-relu-in-deep-learning)
<br>[Conv2D](https://keras.io/layers/convolutional/#conv2d)
<br>[MaxPooling](https://computersciencewiki.org/index.php/Max-pooling_/_Pooling)
<br>[Dense and Dropout](https://www.quora.com/In-Keras-what-is-a-dense-and-a-dropout-layer)
<br>[Softmax](https://sefiks.com/2017/11/08/softmax-as-a-neural-networks-activation-function/)
<br>[Flatten](https://stackoverflow.com/questions/48855804/what-does-flatten-do-in-sequential-model-in-keras)
<br>[Jason Brownlee Digit Recognition Tutorial](https://machinelearningmastery.com/handwritten-digit-recognition-using-convolutional-neural-networks-python-keras/)
<br>[nextjournal Digit Recognition with Keras](https://nextjournal.com/gkoehler/digit-recognition-with-keras)